In [193]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from copy import deepcopy
from datetime import datetime
import json

defaultMissingValue = 999999


def loadLocalJsonDoc(filepath, dataProp=''):
    """
    return deserialised json in dictionary

    Parameters
    ----------
    filepath: file location or buffer.
    dataProp: (optional) specified property to access required data
    """
    output = {}
    with open(file=filepath, mode='r', encoding='utf-8') as f:
        json_load = json.load(f)
        if (dataProp):
            output = json_load[dataProp]
        else:
            output = json_load
    return output


# Import dataset
services = pd.read_csv(
    'Caracterización_de_punto_de_servicio_-_Aurora_Office_-_Ronda_2_-_all_versions_-_False_-_2023-10-26-22-21-20.csv', sep=';', index_col=False)

# Replace "other in organizations"
services.loc[services['organizacionimplementadora'] == "otra",
             'organizacionimplementadora'] = services['cualotraorganizacionimp']
services.loc[services['organizacionprincipal'] == "otra",
             'organizacionprincipal'] = services['cualotraorganizacionprin']

# Fill  missing values of variables of attended separated children
common = services['nnanoacompanados'].isnull()
common1 = services['nnaseparados'].isnull()
condition = [(services['numeronna'] == 0) & (common)]
condition1 = [(services['numeronna'] == 0) & (common)]
fill_with = ['no']

services['nnanoacompanados'] = np.select(
    condition, fill_with, default=services['nnanoacompanados'])
services['nnaseparados'] = np.select(
    condition1, fill_with, default=services['nnaseparados'])

# Fill  missing values
#services = services.fillna(defaultMissingValue)

# rename variables
newColumns = loadLocalJsonDoc("defaults/rename_columns.json")
services_carto = services.rename(columns=newColumns)

In [194]:
def toUnixTimestamp(time, format: str = "%d/%m/%Y"):
    start = datetime(1970, 1, 1)
    target = datetime.strptime(time, format)
    in_seconds = (target - start).total_seconds()
    in_milliseconds = int(in_seconds) * 1000
    return in_milliseconds


def codifyServices(value: str, values_dict: dict[str, int], otherValue: str):
    if (type(value) == float or type(value) == int):
        return otherValue
    raw_values = value.split(" ")
    output = []
    for value in raw_values:
        try:
            codedValue = values_dict[value]
            output.append(str(codedValue))
        except Exception as e:
            output.append(otherValue)

    return "|".join(output)


def processColumn(dfColumn: Series, values_dict: dict[int, str], other_value: str):
    reversed_values_dict = dict([(x[1], x[0]) for x in values_dict.items()])
    return dfColumn.apply(lambda x: codifyServices(x, reversed_values_dict, other_value))


def processMultValueColumns(df: DataFrame, columnObjectsList: list[dict]):
    """
    df: DataFrame object
    columnsObjectsList: list of column object
    columnObject: dictionary {"target_column": str, "output_column": str, values_dict: dict, other_value: str}

    return DataFrame Object
    """
    for columnObject in columnObjectsList:
        try:
            target_column = columnObject["target_column"]
            output_column = columnObject["output_column"]
            values_dict = columnObject["values_dict"]
            other_value = str(columnObject["other_value"])
            df[output_column] = processColumn(
                df[target_column], values_dict, other_value)
        except Exception as e:
            print(e)
            continue
    return df


def exportToFile(df: DataFrame, fileType: str, exportName: str):
    """ 
    df -> Pandas DataFrame object
    fileType -> Either "csv" or "json"
    exportName -> File location
    """
    if (fileType == "csv"):
        name = f"{exportName}.csv"
        df.to_csv(name)
        print(f"data export to {name}")
    else:
        name = f"{exportName}.json"
        df.to_json(name, orient="records")
        print(f"data export to {name}")

In [195]:
# parsing date field into unix timestamp
services_carto["timeunix"] = services_carto["fecha"].apply(
    lambda x: toUnixTimestamp(time=x, format="%Y-%m-%d"))

In [165]:
def codifyServices(value: str, values_dict: dict[str, int], otherValue: str):
    if (type(value) == float or type(value) == int):
        return otherValue
    raw_values = value.split(" ")
    output = []
    for value in raw_values:
        try:
            codedValue = values_dict[value]
            output.append(str(codedValue))
        except Exception as e:
            output.append(otherValue)

    return "|".join(output)


def processColumn(dfColumn: Series, values_dict: dict[int, str], other_value: str):
    reversed_values_dict = dict([(x[1], x[0]) for x in values_dict.items()])
    return dfColumn.apply(lambda x: codifyServices(x, reversed_values_dict, other_value))


def processMultValueColumns(df: DataFrame, columnObjectsList: list[dict]):
    """
    df: DataFrame object
    columnsObjectsList: list of column object
    columnObject: dictionary {"target_column": str, "output_column": str, values_dict: dict, other_value: str}

    return DataFrame Object
    """
    for columnObject in columnObjectsList:
        try:
            target_column = columnObject["target_column"]
            output_column = columnObject["output_column"]
            values_dict = columnObject["values_dict"]
            other_value = str(columnObject["other_value"])
            df[output_column] = processColumn(
                df[target_column], values_dict, other_value)
        except Exception as e:
            print(e)
            continue
    return df


def exportToFile(df: DataFrame, fileType: str, exportName: str):
    """ 
    df -> Pandas DataFrame object
    fileType -> Either "csv" or "json"
    exportName -> File location
    """
    if (fileType == "csv"):
        name = f"{exportName}.csv"
        df.to_csv(name)
        print(f"data export to {name}")
    else:
        name = f"{exportName}.json"
        df.to_json(name, orient="records")
        print(f"data export to {name}")

In [166]:
# serv_tipo (separating by pipe symbol and categorized with number)
codify_dict = loadLocalJsonDoc("defaults/codification_dict.json")
services_dict = codify_dict["services_dict"]

# re structure variable cuenta_con
cuenta_con_dict = codify_dict["cuenta_con_dict"]

# re structure variable children services (cual_serv1)
cual_serv1_dict = codify_dict["cual_serv1_dict"]

# re structure variable women services (cual_ser_2)
cual_ser_2_dict = codify_dict["cual_ser_2_dict"]

# re structure variable data storage (almacenamientoregistros)
registro_dict = codify_dict["registro_dict"]

# variable funding
financ_dict = codify_dict["financ_dict"]

#  variable challenges
reto_dict = codify_dict["reto_dict"]

# variable lenguages
idio_dict = codify_dict["idio_dict"]

# variable medios
medio_dict = codify_dict["medio_dict"]

values = [
    {
        "target_column": "serv_tipo",
        "output_column": "serv_tipo1",
        "values_dict": services_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "cuenta_con",
        "output_column": "cuenta_c_1",
        "values_dict": cuenta_con_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "cual_serv1",
        "output_column": "cual_ser_1",
        "values_dict": cual_serv1_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "cual_ser_2",
        "output_column": "cual_ser_3",
        "values_dict": cual_ser_2_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "almacenamientoregistros",
        "output_column": "almacenamientoregistros_",
        "values_dict": cual_ser_2_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "financiamiento",
        "output_column": "financb",
        "values_dict": financ_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "princ_reto",
        "output_column": "princ_re_1",
        "values_dict": reto_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "idioma_ent",
        "output_column": "idioma_e_1",
        "values_dict": idio_dict,
        "other_value": defaultMissingValue
    },
    {
        "target_column": "medios_bri",
        "output_column": "medios_b_1",
        "values_dict": medio_dict,
        "other_value": defaultMissingValue
    },
]

In [167]:
output_df = processMultValueColumns(services_carto, values)

In [96]:
exportToFile(output_df, "csv", "output")

data export to output.csv


# daily report of surveys


In [196]:
import datetime

services.fecha = pd.to_datetime(services.fecha)

In [197]:
# totals
survey_counts = services.groupby(
    'encuesta').organizacionprincipal.count().reset_index()
survey_counts = survey_counts.rename(
    columns={"organizacionprincipal": "total_encuestas"})
survey_counts

# totals by organization
unpivoted = services.groupby(['encuesta', 'fecha'])[
    'pais'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='fecha',
    index='encuesta',
    values='pais')
pivoted

fecha,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-26
encuesta,,,,,,,,
andresmauricioclavijoabril,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
glenisyurleybarraganjaimes,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
luisamonserratsolisriveros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
luiscarlosmejiafallas,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
luisricardosolermadrid,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
otro,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
rosakatherinecasanovasilva,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
sheilacristinamunozchoque,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN
valentinaalondramedinamedina,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN


In [215]:
# capacity vs attentions
services['servicios'] = services['servicios'].str.split(' ')
prueba = services.explode('servicios')


In [199]:
prueba = prueba[['_coordenadas_latitude',
                '_coordenadas_longitude',
                 'servicios',
                 'organizacionprincipal',
                 'organizacionimplementadora',
                 'pais',
                 'punto_reporte',
                 '_uuid',
                 'diarioalimentacionynutricion',
                 'ayeralimentacionynutricion',
                 'promalimentacionynutricion',
                 'diarioalojamientotemporal',
                 'ayeralojamientotemporal',
                 'promalojamientotemporal',
                 'diarioapoyooayudapsicosocial',
                 'ayerapoyooayudapsicosocial',
                 'promapoyooayudapsicosocial',
                 'diarioasesorialegal',
                 'ayerasesorialegal',
                 'promasesorialegal',
                 'diarioeducacion',
                 'ayereducacion',
                 'promeducacion',
                 'diarioserviciosproteccion',
                 'ayerserviciosproteccion',
                 'promserviciosproteccion',
                 'diariorestablecimientodecontactofamiliar',
                 'ayerrestablecimientodecontactofamiliar',
                 'promrestablecimientodecontactofamiliar',
                 'diariosaludprimerosauxiliosyatencionmedica',
                 'ayersaludprimerosauxiliosyatencionmedica',
                 'promsaludprimerosauxiliosyatencionmedica',
                 'diarioaguapotable',
                 'ayeraguapotable',
                 'promaguapotable',
                 'diariosaneamiento',
                 'ayersaneamiento',
                 'promsaneamiento',
                 'diariohigiene',
                 'ayerhigiene',
                 'promhigiene',
                 'diariotransportehumanitario',
                 'ayertransportehumanitario',
                 'promtransportehumanitario',
                 'diariotransferenciasmonetarias',
                 'ayertransferenciasmonetarias',
                 'promtransferenciasmonetarias',
                 'diariootro',
                 'ayerotro',
                 'promotro',
                 'diariootro1',
                 'ayerotro1',
                 'promotro1',
                 ]].reset_index()

In [200]:
prueba["capacidad"] = " "
prueba["ayer"] = " "
prueba["semana_pasada"] = " "

In [201]:
# Look up function

def fillColumnWithLookup(df: DataFrame, values: list[tuple[str, str]], lookup_column: str, target_column: str):
    """ 
    returns DataFrame with new columns filled with lookup parameters

    Parameters
    ----------
    df: DataFrame
    values: list of 2-dimensional tuple. First is testing value.
    Second is the output columns to which to place values that match the first.
    lookup_column: a string of the field or column or dict property to which test the testing value
    target_column: a string of a field or column or dict property to get the required value
    """
    working_df = deepcopy(df)
    for (value, output_column) in values:
        working_df.loc[(working_df[lookup_column] == value),
                       target_column] = working_df[output_column]
    return working_df


def processColumnFill(df: DataFrame, columnValues: list[dict]):
    """ 
    returns DataFrame with new columns filled with lookup parameters

    Parameters
    ----------
    df: DataFrame
    columnValues: list of Objects having:
        values: list of 2-dimensional tuple. First is testing value.
        Second is the output columns to which to place values that match the first.
        lookup_column: a string of the field or column or dict property to which test the testing value
        target_column: a string of a field or column or dict property to get the required value
    """
    working_df = deepcopy(df)
    for column in columnValues:
        lookup_column = column["lookup_column"]
        target_column = column["target_column"]
        values = column["values"]
        working_df = fillColumnWithLookup(
            df=working_df, values=values, target_column=target_column, lookup_column=lookup_column)
    return working_df

In [202]:
diarValues = [
    ('alimentacionynutricion', 'diarioalimentacionynutricion'),
    ('alojamientotemporal', 'diarioalojamientotemporal'),
    ('acompañamientoayudapsicosocial', 'diarioapoyooayudapsicosocial'),
    ('aguapotable', 'diarioaguapotable'),
    ('asesorialegal', 'diarioasesorialegal'),
    ('educacion', 'diarioeducacion'),
    ('higiene', 'diariohigiene'),
    ('restablecimientodecontactofamiliar',
     'diariorestablecimientodecontactofamiliar'),
    ('saludprimerosauxiliosyatencionmedica',
     'diariosaludprimerosauxiliosyatencionmedica'),
    ('saneamiento', 'diariosaneamiento'),
    ('serviciosdeproteccion', 'diarioserviciosproteccion'),
    ('transferencias', 'diariotransferenciasmonetarias'),
    ('transportehumanitario', 'diariotransportehumanitario'),
    ('otro', 'diariootro'),
]
ayerValues = [
    ('alimentacionynutricion', 'ayeralimentacionynutricion'),
    ('alojamientotemporal', 'ayeralojamientotemporal'),
    ('acompañamientoayudapsicosocial', 'ayerapoyooayudapsicosocial'),
    ('aguapotable', 'ayeraguapotable'),
    ('asesorialegal', 'ayerasesorialegal'),
    ('educacion', 'ayereducacion'),
    ('higiene', 'ayerhigiene'),
    ('restablecimientodecontactofamiliar',
     'ayerrestablecimientodecontactofamiliar'),
    ('saludprimerosauxiliosyatencionmedica',
     'ayersaludprimerosauxiliosyatencionmedica'),
    ('saneamiento', 'ayersaneamiento'),
    ('serviciosdeproteccion', 'ayerserviciosproteccion'),
    ('transferencias', 'ayertransferenciasmonetarias'),
    ('transportehumanitario', 'ayertransportehumanitario'),
    ('otro', 'ayerotro'),
]
promValues = [
    ('alimentacionynutricion', 'promalimentacionynutricion'),
    ('alojamientotemporal', 'promalojamientotemporal'),
    ('acompañamientoayudapsicosocial', 'promapoyooayudapsicosocial'),
    ('aguapotable', 'promaguapotable'),
    ('asesorialegal', 'promasesorialegal'),
    ('educacion', 'promeducacion'),
    ('higiene', 'promhigiene'),
    ('restablecimientodecontactofamiliar',
     'promrestablecimientodecontactofamiliar'),
    ('saludprimerosauxiliosyatencionmedica',
     'promsaludprimerosauxiliosyatencionmedica'),
    ('saneamiento', 'promsaneamiento'),
    ('serviciosdeproteccion', 'promserviciosproteccion'),
    ('transferencias', 'promtransferenciasmonetarias'),
    ('transportehumanitario', 'promtransportehumanitario'),
    ('otro', 'promotro'),
]


columnValues = [
    {
        "target_column": "capacidad",
        "lookup_column": "servicios",
        "values": diarValues
    },
    {
        "target_column": "ayer",
        "lookup_column": "servicios",
        "values": ayerValues
    },
    {
        "target_column": "semana_pasada",
        "lookup_column": "servicios",
        "values": promValues
    }
]

In [203]:
prueba = processColumnFill(df=prueba, columnValues=columnValues)

In [204]:
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="colchane",
           value="Colchane")
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="villa del rosario",
           value="Villa del Rosario")
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="lospatios",
           value="Los Patios") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="arica",
           value="Arica") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="iquique",
           value="Iquique") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="dtlobito",
           value="DT Lobito") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="loschiles",
           value="Los Chiles") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="necocli",
           value="Necoclí") 
prueba['punto_reporte']=prueba['punto_reporte'].replace(to_replace="ipiales",
           value="Ipiales") 

prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="unicef",
           value="Unicef")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="acnur",
           value="Acnur")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="cruzroja",
           value="Cruz Roja")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="serviciojesuitaparamigrantes",
           value="Servicio Jesuita para Migrantes")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="worldvision",
           value="World Vision")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="samaritanspurse",
           value="Samaritans Purse")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="ayudaenaccion",
           value="Ayuda en Acción")
prueba['organizacionprincipal']=prueba['organizacionprincipal'].replace(to_replace="hias",
           value="Hias")


prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="hias",
           value="Hias")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="worldvision",
           value="World Vision")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="cruzroja",
           value="Cruz Roja")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="serviciojesuitaparamigrantes",
           value="Servicio Jesuita para Migrantes")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="samaritanspurse",
           value="Samaritans Purse")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="ayudaenaccion",
           value="Ayuda en Acción")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="hias",
           value="Hias")
prueba['organizacionimplementadora']=prueba['organizacionimplementadora'].replace(to_replace="unicef",
           value="Unicef")

prueba['servicios']=prueba['servicios'].replace(to_replace="aguapotable",
           value="Agua Potable")
prueba['servicios']=prueba['servicios'].replace(to_replace="alimentacionynutricion",
           value="Alimentación y Nutrición")
prueba['servicios']=prueba['servicios'].replace(to_replace="alojamientotemporal",
           value="Alojamiento Temporal")
prueba['servicios']=prueba['servicios'].replace(to_replace="higiene",
           value="Higiene")
prueba['servicios']=prueba['servicios'].replace(to_replace="saludprimerosauxiliosyatencionmedica",
           value="Salud, Primeros Auxilios y Atención Médica")
prueba['servicios']=prueba['servicios'].replace(to_replace="saneamiento",
           value="Saneamiento")
prueba['servicios']=prueba['servicios'].replace(to_replace="transportehumanitario",
           value="Transporte Humanitario")
prueba['servicios']=prueba['servicios'].replace(to_replace="asesorialegal",
           value="Asesoría Legal")
prueba['servicios']=prueba['servicios'].replace(to_replace="educacion",
           value="Educación")
prueba['servicios']=prueba['servicios'].replace(to_replace="serviciosdeproteccion",
           value="Servicios de Protección")
prueba['servicios']=prueba['servicios'].replace(to_replace="otro",
           value="Otro")
prueba['servicios']=prueba['servicios'].replace(to_replace="acompañamientoayudapsicosocial",
           value="Acompañamiento y/o Ayuda Psicosocial")
prueba['servicios']=prueba['servicios'].replace(to_replace="transferencias",
           value="Transferencias")
prueba['servicios']=prueba['servicios'].replace(to_replace="restablecimientodecontactofamiliar",
           value="Restablecimiento de Contacto Familiar")


In [205]:
newColumns = {'_uuid': 'submission_id',
              'servicios': 'servicio',
              'semana_pasada': 'semana+anterior',
              'ayer': 'promedio+ayer',
              '_coordenadas_latitude': 'observation_lat',
              '_coordenadas_longitude': 'observation_lon',
              'punto_reporte': 'Región'
              }

prueba = prueba.rename(columns=newColumns)

In [206]:
wide = prueba[['submission_id',
               'servicio',
               'capacidad',
               'semana+anterior',
               'promedio+ayer',
               'observation_lat',
               'observation_lon',
               'Región',
               'organizacionprincipal',
               'organizacionimplementadora',

               ]].reset_index()

In [208]:
wide1 = wide.dropna(subset=['servicio']).reset_index(drop=True)

In [209]:
powerbi = services[[ '_uuid',
                 'fecha',
                 '_coordenadas_latitude',
                 '_coordenadas_longitude',
                 'punto_reporte',
                 'organizacionprincipal', 
                 'organizacionimplementadora',
                 'pais'	,
                 'punto_reporte' ,
                 'faltadepersonal',
                 'faltadeinsumos',	
                 'condicionesinfraestructura',
                 'capacidadsobrepasada']].reset_index()


powerbi['pais']=powerbi['pais'].replace(to_replace="chile",
           value="Chile")
powerbi['pais']=powerbi['pais'].replace(to_replace="costarica",
           value="Costa Rica")
powerbi['pais']=powerbi['pais'].replace(to_replace="colombia",
           value="Colombia") 

powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="colchane",
           value="Colchane")
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="villa del rosario",
           value="Villa del Rosario")
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="lospatios",
           value="Los Patios") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="arica",
           value="Arica") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="iquique",
           value="Iquique") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="dtlobito",
           value="DT Lobito") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="loschiles",
           value="Los Chiles") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="necocli",
           value="Necoclí") 
powerbi['punto_reporte']=powerbi['punto_reporte'].replace(to_replace="ipiales",
           value="Ipiales") 

powerbi['faltadepersonal']=powerbi['faltadepersonal'].replace(to_replace="si",
           value="Sí")
powerbi['faltadepersonal']=powerbi['faltadepersonal'].replace(to_replace="no",
           value="No")

powerbi['faltadeinsumos']=powerbi['faltadeinsumos'].replace(to_replace="si",
           value="Sí")
powerbi['faltadeinsumos']=powerbi['faltadeinsumos'].replace(to_replace="no",
           value="No")

powerbi['condicionesinfraestructura']=powerbi['condicionesinfraestructura'].replace(to_replace=1,
           value="Totalmente Inadecuada")
powerbi['condicionesinfraestructura']=powerbi['condicionesinfraestructura'].replace(to_replace=2,
           value="Inadecuada")
powerbi['condicionesinfraestructura']=powerbi['condicionesinfraestructura'].replace(to_replace=3,
           value="Ni Inadecuada/ Ni Adecuada")
powerbi['condicionesinfraestructura']=powerbi['condicionesinfraestructura'].replace(to_replace=4,
           value="Adecuada")
powerbi['condicionesinfraestructura']=powerbi['condicionesinfraestructura'].replace(to_replace=5,
           value="Totalmente Adecuada")

powerbi['capacidadsobrepasada']=powerbi['capacidadsobrepasada'].replace(to_replace="ocasionalmente",
           value="Ocasionalmente")
powerbi['capacidadsobrepasada']=powerbi['capacidadsobrepasada'].replace(to_replace="nunca",
           value="Nunca")
powerbi['capacidadsobrepasada']=powerbi['capacidadsobrepasada'].replace(to_replace="raramente",
           value="Raramente")
powerbi['capacidadsobrepasada']=powerbi['capacidadsobrepasada'].replace(to_replace="frecuentemente",
           value="Frecuentemente")
powerbi['capacidadsobrepasada']=powerbi['capacidadsobrepasada'].replace(to_replace="siempre",
           value="Siempre")

powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="unicef",
           value="Unicef")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="acnur",
           value="Acnur")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="cruzroja",
           value="Cruz Roja")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="serviciojesuitaparamigrantes",
           value="Servicio Jesuita para Migrantes")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="worldvision",
           value="World Vision")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="samaritanspurse",
           value="Samaritans Purse")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="ayudaenaccion",
           value="Ayuda en Acción")
powerbi['organizacionprincipal']=powerbi['organizacionprincipal'].replace(to_replace="hias",
           value="Hias")


powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="hias",
           value="Hias")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="worldvision",
           value="World Vision")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="cruzroja",
           value="Cruz Roja")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="serviciojesuitaparamigrantes",
           value="Servicio Jesuita para Migrantes")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="samaritanspurse",
           value="Samaritans Purse")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="ayudaenaccion",
           value="Ayuda en Acción")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="hias",
           value="Hias")
powerbi['organizacionimplementadora']=powerbi['organizacionimplementadora'].replace(to_replace="unicef",
           value="Unicef")




In [210]:
powerbi

,index,_uuid,fecha,_coordenadas_latitude,_coordenadas_longitude,punto_reporte,organizacionprincipal,organizacionimplementadora,pais,punto_reporte,faltadepersonal,faltadeinsumos,condicionesinfraestructura,capacidadsobrepasada
0,0,fe511e3c-71c1-4eaa-8366-e0543dd34bd9,2023-10-16,-19.275246,-68.621534,Colchane,Delegación Presidencial Regional de Tarapacá,Productora 370,Chile,Colchane,Sí,Sí,Ni Inadecuada/ Ni Adecuada,Ocasionalmente
1,1,ba2fd2d1-b196-495d-b0ad-d651ca780911,2023-10-16,-19.275197,-68.621440,Colchane,Ministerio de Desarrollo Social y Familia,Fundación Hogar de Cristo,Chile,Colchane,Sí,Sí,Totalmente Adecuada,Ocasionalmente
2,2,e71686ab-2436-4dde-b3ef-e2d92e3044d3,2023-10-17,-18.481970,-70.303048,Arica,Unicef,World Vision,Chile,Arica,Sí,Sí,Adecuada,Nunca
3,3,aba85c33-15b3-4776-89c5-99e71bc7e675,2023-10-17,-18.481886,-70.302974,Arica,Acnur,World Vision,Chile,Arica,Sí,No,Adecuada,Ocasionalmente
4,4,3e2e270e-47ac-4217-8742-d62910630e1d,2023-10-18,7.817663,-72.456640,Villa del Rosario,Cruz Roja,Cruz Roja,Colombia,Villa del Rosario,Sí,Sí,Adecuada,Raramente
5,5,70b3fc35-de88-47ba-89f9-9db281b73092,2023-10-18,-18.475863,-70.305139,Arica,Servicio Jesuita para Migrantes,Servicio Jesuita para Migrantes,Chile,Arica,Sí,Sí,Adecuada,Ocasionalmente
6,6,0d0c8752-dc03-4541-9aeb-c63e2d07b7d8,2023-10-18,-18.482151,-70.303125,Arica,World Vision,World Vision,Chile,Arica,No,Sí,Adecuada,Nunca
7,7,260990e1-2962-4891-b1d0-12accf2b0c6b,2023-10-18,7.691837,-72.599467,Los Patios,Samaritans Purse,Fundscion mujer y futuro en alianza con oxfam ...,Colombia,Los Patios,No,No,Adecuada,Frecuentemente
8,8,598bd59a-85b3-4ef0-851c-fc5b49f5602e,2023-10-18,-20.220662,-70.137795,Iquique,Acnur,Fundación Madre Josefa,Chile,Iquique,Sí,No,Adecuada,Frecuentemente
9,9,fc06965b-46e8-4583-b9f2-2b53f6cdb058,2023-10-18,7.691672,-72.599563,Los Patios,Samaritans Purse,Samaritans Purse,Colombia,Los Patios,No,No,Totalmente Adecuada,Nunca


In [211]:
newColumns = { '_uuid' : 'submission_id',
               '_coordenadas_latitude' : 'observation_lat',
               '_coordenadas_longitude' : 'observation_lon',
               'punto_reporte' : 'Región'
               }

powerbi = powerbi.rename(columns=newColumns)

In [212]:
powerbi = powerbi[powerbi['capacidadsobrepasada'].notna()]

In [214]:
output_df = processMultValueColumns(wide1, values)
exportToFile(output_df, "csv", "bd_serv_prem_wide")

output_pw = processMultValueColumns(powerbi, values)
exportToFile(output_pw, "csv", "powerbi")

'serv_tipo'
'cuenta_con'
'cual_serv1'
'cual_ser_2'
'almacenamientoregistros'
'financiamiento'
'princ_reto'
'idioma_ent'
'medios_bri'
data export to bd_serv_prem_wide.csv
'serv_tipo'
'cuenta_con'
'cual_serv1'
'cual_ser_2'
'almacenamientoregistros'
'financiamiento'
'princ_reto'
'idioma_ent'
'medios_bri'
data export to powerbi.csv
